In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from dateutil import parser
import math
import requests
import json

pd.options.plotting.backend = "plotly"

%matplotlib inline

In [ ]:
from_date = '2022-02-14T16:00:00'
to_date = '2022-02-14T18:00:00'

In [ ]:
const_coeff = -0.022223
HR_coeff = 0.000281
Weight_coeff = 0.000081
Acc_coeff = 0.000002

Weight = 70

In [ ]:
url = 'https://api.sci.appadem.in/data/accel?from=' + from_date + '&to=' + to_date
response = requests.get(url).text
acc_raw = json.loads(response)

In [ ]:
url = 'https://api.sci.appadem.in/data/hr?from=' + from_date + '&to=' + to_date
response = requests.get(url).text
hr_raw = json.loads(response)

In [ ]:
acc_raw[0]

In [ ]:
rows = []
for d in acc_raw:
    x = d['x']
    y = d['y']
    z = d['z']
    a = np.sqrt(x ** 2 + y ** 2 + z ** 2)
    rows.append([pd.to_datetime(d['t']), x, y, z, a])

acc = pd.DataFrame(rows, columns=['time', 'x', 'y', 'z', 'a'])
acc = acc.set_index('time')

In [ ]:
rows = []
for d in hr_raw:
    rows.append([pd.to_datetime(d['t']), d['hr']])

hr = pd.DataFrame(rows, columns=['time', 'value'])
hr = hr.set_index('time')

In [ ]:
acc.head()

In [ ]:
# Plot accel x,y,z over time
hr.sort_values(by='time').plot(x='time', y=['value'], )

In [ ]:
# Plot accel x,y,z over time

acc.plot(x='time', y=['a'])

In [ ]:
# acc['mean'] = acc['a'].rolling(10*60).sum()/9.82
acc['mean'] = acc.resample('1Min').sum()['a']
acc.plot(y=['mean'])

In [ ]:
hr['mean'] = hr['value'].rolling(60).mean()

hr.plot(x='time', y=['mean', 'value'])

In [ ]:
acc['time']

In [ ]:
# acc.groupby([acc['time'].dt.minute]).mean()
acc_minutes = acc.resample('1Min')
hr_minutes = hr.resample('1Min')
result = acc_minutes.sum()
result['count'] = acc_minutes['a'].count()
result['hr'] = hr_minutes.mean()['value']
result['acc'] = result['a']/9.82
result['aee_min_kg'] = const_coeff + result['acc']*Acc_coeff + Weight * Weight_coeff + result['hr'] * HR_coeff
result['aee_min'] = result['aee_min_kg'] * Weight
result


In [ ]:
result.cumsum().plot(y=['aee_min'])

In [ ]:
acc['x'] = new['x']

In [ ]:
acc.resample('1Min').count()['a'].plot()